In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from config import get_chrome_driver
from utils import ProductSerializer

In [6]:
class KilimalSeleniumScrapper():
    """
    search_parameters => dict object {q:query,page:page_number}
    self.url = 'https://www.kilimall.co.ke/new/commoditysearch?'
    """
    def __init__(self):
        self.items_found = 0
        self.pages_found = 0
        self.title = ''
        self.driver = get_chrome_driver(incognito=True)
        self.dataframe = pd.DataFrame(columns=["product_name", "item_link","image_src","price","prev_price", "item_review", "rating", 'per_discount'])
    def call(self,filter_parameters,page_number='page_1'):
        parameters='&'.join(['{}={}'.format(key,filter_parameters[key]) for key in filter_parameters.keys()])
        print(self.url+parameters)
        self.driver.get(self.url+parameters)
        html_string = self.driver.page_source
        soup= BeautifulSoup(html_string, 'lxml')
        self.title = soup.title.text
        ele_result= soup.body.find_all(class_="el-col el-col-6") 
        for i, article in enumerate(ele_result):
            item_link,prev_price, product_name, review, image_path,rating,price, per_discount =None, None, None, 0, None, None, None, '0%'
            try:
                price_ele = article.find(class_='wordwrap-price')
                price = price_ele.find('span').text
                prev_price =  price_ele.find(class_='twoksh').text
                item_link = ('https://www.kilimall.co.ke/{}').format(article.a['href'])
                product_name = article.find(class_='wordwrap').text
                image_path=article.img['src']
                per_discount = article.find(class_='greenbox').text
                review = article.find(class_='sixtwo').text 
                if page_number=='page_1':
                    self.result_found = soup.body.find(class_='results').text.split(' ')[1]
                    pages_found = soup.body.find(class_='leftpaging').text.split('/')[-1]
                    self.pages_found = int(pages_found)
            except:
                pass
            if prev_price == None:
                prev_price=price
            item_data = ProductSerializer(name=product_name,item_link=item_link, 
                                          image_path=image_path, price=price, 
                                          prev_price=prev_price, item_reviews=review,rating=rating, per_discount=per_discount)
            page_dataframe = pd.DataFrame(item_data,index=[0])        
            self.dataframe = pd.concat([self.dataframe,page_dataframe], ignore_index=True)
    def get_result(self, filter_parameters, max_number_pages=5):
        self.call(filter_parameters)
        for i in range(2,max_number_pages+1):
            if i<=self.pages_found:
                filter_parameters['page']=i
                self.call(filter_parameters,page_number='page_{}'.format(i))
        self.driver.quit()        
        return {'title':self.title,'items_found':self.result_found,'dataframe':self.dataframe}          

In [9]:
kilimal = KilimalSeleniumScrapper()
result = kilimal.get_result({'q':'men+watches'},max_number_pages=3)

/home/ojay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


https://www.kilimall.co.ke/new/commoditysearch?q=men+watches
https://www.kilimall.co.ke/new/commoditysearch?q=men+watches&page=2
https://www.kilimall.co.ke/new/commoditysearch?q=men+watches&page=3


In [2]:
get_chrome_driver??